In [1]:
import pandas as pd
from datetime import datetime
from datetime import datetime
from decimal import Decimal
import random

from strat_backtest.base.gen_trades import GenTrades, TradingConfig, RiskConfig

In [2]:
df = pd.read_parquet("../data/AAPL.parquet")
df

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,AAPL,71.72,72.78,71.47,72.72,135480400
2020-01-03,AAPL,71.94,72.77,71.78,72.01,146322800
2020-01-06,AAPL,71.13,72.62,70.88,72.58,118387200
2020-01-07,AAPL,72.59,72.85,72.02,72.24,108872000
2020-01-08,AAPL,71.94,73.71,71.94,73.40,132079200
...,...,...,...,...,...,...
2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400
2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300
2025-04-21,AAPL,193.27,193.80,189.81,193.16,46742500


In [13]:
df1 = df.loc["2025-04-04":, :].reset_index()
df1

,Date,Ticker,Open,High,Low,Close,Volume
0,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900
1,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300
2,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900
4,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000
5,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900
7,2025-04-15,AAPL,201.86,203.51,199.80,202.14,51343900
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400
9,2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300


In [8]:
df1.columns = [col.lower() for col in df1.columns]
df1

,date,ticker,open,high,low,close,volume
0,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900
1,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300
2,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900
4,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000
5,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900
7,2025-04-15,AAPL,201.86,203.51,199.80,202.14,51343900
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400
9,2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300


In [9]:
df1["entry_signal"] = [
    "wait",
    "buy",
    "buy",
    "wait",
    "wait",
    "buy",
    "wait",
    "wait",
    "wait",
    "buy",
]
df1["exit_signal"] = [
    "sell",
    "wait",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
]
df1

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,sell
1,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300,buy,wait
2,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500,buy,wait
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
4,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000,wait,sell
5,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,buy,wait
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,wait,wait
7,2025-04-15,AAPL,201.86,203.51,199.80,202.14,51343900,wait,sell
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400,wait,wait
9,2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300,buy,wait


In [10]:
df1.loc[df1["entry_signal"] == df1["exit_signal"], :]

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,wait,wait
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400,wait,wait


In [11]:
df1.to_parquet("../tests/data/sample_gen_trades.parquet", index=False)

temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,sell
1,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300,buy,wait
2,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500,buy,wait
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
4,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000,wait,sell
5,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,buy,wait
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,wait,wait
7,2025-04-15,AAPL,201.86,203.51,199.80,202.14,51343900,wait,sell
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400,wait,wait
9,2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300,buy,wait


In [11]:
all(isinstance(dt, datetime) for dt in temp["date"])

True

In [12]:
temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,sell
1,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300,buy,wait
2,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500,buy,wait
3,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
4,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000,wait,sell
5,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,buy,wait
6,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,wait,wait
7,2025-04-15,AAPL,201.86,203.51,199.80,202.14,51343900,wait,sell
8,2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400,wait,wait
9,2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300,buy,wait


In [14]:
temp.loc[temp["entry_signal"]=="buy", :]

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
1,2025-04-07,AAPL,177.2,194.15,174.62,181.46,160466300,buy,wait
2,2025-04-08,AAPL,186.7,190.34,169.21,172.42,120859500,buy,wait
5,2025-04-11,AAPL,186.1,199.54,186.06,198.15,87435900,buy,wait
9,2025-04-17,AAPL,197.2,198.83,194.42,196.98,51334300,buy,wait


In [3]:
temp.iloc[-1, :].to_dict()

{'date': Timestamp('2025-04-14 00:00:00'),
 'ticker': 'AAPL',
 'open': 211.16,
 'high': 212.66,
 'low': 200.9,
 'close': 202.25,
 'volume': 101352900.0,
 'entry_signal': 'buy',
 'exit_signal': 'wait'}

In [109]:
a = temp.loc[temp["date"] == "2025-04-08", :].to_dict(orient="records")[0]
a

{'date': Timestamp('2025-04-08 00:00:00'),
 'ticker': 'AAPL',
 'open': 186.46,
 'high': 190.09,
 'low': 168.99,
 'close': 172.19,
 'volume': 120859500.0,
 'entry_signal': 'buy',
 'exit_signal': 'wait'}

In [117]:
for record in temp.itertuples(name=None, index=False):
    print(record)

(Timestamp('2025-04-01 00:00:00'), 'AAPL', 219.81, 223.68, 218.9, 223.19, 36412700.0, 'wait', 'sell')
(Timestamp('2025-04-02 00:00:00'), 'AAPL', 221.32, 225.19, 221.02, 223.89, 35905900.0, 'buy', 'wait')
(Timestamp('2025-04-03 00:00:00'), 'AAPL', 205.54, 207.49, 201.25, 203.19, 103419000.0, 'buy', 'wait')
(Timestamp('2025-04-04 00:00:00'), 'AAPL', 193.64, 199.62, 187.09, 188.13, 125910900.0, 'wait', 'wait')
(Timestamp('2025-04-07 00:00:00'), 'AAPL', 176.97, 193.9, 174.39, 181.22, 160466300.0, 'wait', 'sell')
(Timestamp('2025-04-08 00:00:00'), 'AAPL', 186.46, 190.09, 168.99, 172.19, 120859500.0, 'buy', 'wait')
(Timestamp('2025-04-09 00:00:00'), 'AAPL', 171.72, 200.35, 171.66, 198.59, 184395900.0, 'wait', 'wait')
(Timestamp('2025-04-10 00:00:00'), 'AAPL', 188.82, 194.52, 182.76, 190.17, 121880000.0, 'wait', 'sell')
(Timestamp('2025-04-11 00:00:00'), 'AAPL', 185.86, 199.28, 185.82, 197.89, 87435900.0, 'wait', 'wait')
(Timestamp('2025-04-14 00:00:00'), 'AAPL', 211.16, 212.66, 200.9, 202.25

In [119]:
df1 = df.reset_index()
df1

,Date,Ticker,Open,High,Low,Close,Volume
0,2020-01-02,AAPL,71.72,72.78,71.47,72.72,135480400.0
1,2020-01-03,AAPL,71.94,72.77,71.78,72.01,146322800.0
2,2020-01-06,AAPL,71.13,72.62,70.88,72.58,118387200.0
3,2020-01-07,AAPL,72.59,72.85,72.02,72.24,108872000.0
4,2020-01-08,AAPL,71.94,73.71,71.94,73.40,132079200.0
...,...,...,...,...,...,...,...
1345,2025-05-09,AAPL,198.74,200.28,197.28,198.27,36453900.0
1346,2025-05-12,AAPL,210.97,211.27,206.75,210.79,63775800.0
1347,2025-05-13,AAPL,210.43,213.40,209.00,212.93,51909300.0
1348,2025-05-14,AAPL,212.43,213.94,210.58,212.33,49325800.0


In [18]:
from strat_backtest.utils.constants import OpenTrades, Record
from typing import Callable, Any

def cal_trailing_price(
    trigger_trail: float = 0.05,
    step: float | None = None,
    just_testing: str = "testing",
) -> Decimal:
    
    print(locals())

cal_trailing_price()

{'trigger_trail': 0.05, 'step': None, 'just_testing': 'testing'}


In [21]:
223.89 * 1.001

224.11388999999997